In [1]:
import torch
import torch.nn as nn


# Building a generator and discriminator

At PyBooks, you're tasked with working on an automatic text generator to help writers overcome writer's block. By using GANs, or Generative Adversarial Networks, you believe you can create a system where one network, the generator, creates new text while the other network, the discriminator, evaluates its authenticity. To do this, you need to initialize both a generator and discriminator network. These networks will then be trained against each other to create new, believable text.

* Define the Generator class with a linear layer for sequential data and a sigmoid activation function.
* Pass the input through the defined model in the forward() method of the Generator class.
* Define a Discriminator class with the same layers and activation function, taking care when defining the dimensions.

In [3]:
# Define the generator class
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(nn.Linear(seq_length, seq_length), nn.Sigmoid())
    def forward(self, x):
        return self.model(x)

# Define the discriminator networks
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(nn.Linear(seq_length, 1), nn.Sigmoid())
    def forward(self, x):
        return self.model(x)

Great work! You have successfully defined a generator and a discriminator for your GAN. These networks will be the backbone of your text generation system. Now you're ready to move on to the next step, which is to train these networks against each other so they can start generating and evaluating new text!

# Training a GAN model

Your team at PyBooks has made good progress in building the text generator using a Generative Adversarial Network (GAN). You have successfully defined the generator and discriminator networks. Now, it's time to train them. The final step is to generate some fake data and compare it with the real data to see how well your GAN has learned. We have used tensors as an input and the output would try to resemble the input tensors. The team at PyBooks can then use this synthetic data for text analysis as the features will have same relationship as text data.


* Define the loss function for binary classification and the Adam optimizer.
* Train the discriminator by unsqueezing real_data and preventing gradient recalculations.
* Train the generator by calculating the loss and zeroing the gradients.
* Detach the tensor to prevent further computation and print data.

In [16]:
seq_length = 5# Length of each synthetic data sequence
num_sequences = 100 # Total number of sequences generated
num_epochs = 100 #Number of complete passes through the dataset
print_every = 10 #Output display frequency, showing results every 10 epochs

# Generate random synthetic data
# data = torch.rand((num_sequences, seq_length))
data = torch.randint(0, 2, (num_sequences, seq_length)).float()

# Initialize Generator and Discriminator
generator = Generator()
discriminator = Discriminator()


# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer_gen = torch.optim.Adam(generator.parameters(), lr=0.001)
optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=0.001)

**Training Generator and Dicriminator**

In [17]:
for epoch in range(num_epochs):
    for real_data in data:
        # Unsqueezing real_data and prevent gradient recalculations
        real_data = real_data.unsqueeze(0)
        noise = torch.rand((1, seq_length))
        fake_data = generator(noise)
        disc_real = discriminator(real_data)
        disc_fake = discriminator(fake_data.detach())
        loss_disc = criterion(disc_real, torch.ones_like(disc_real)) + criterion(disc_fake, torch.zeros_like(disc_fake))
        optimizer_disc.zero_grad()
        loss_disc.backward()
        optimizer_disc.step()

        # Train the generator
        disc_fake = discriminator(fake_data)
        loss_gen = criterion(disc_fake, torch.ones_like(disc_fake))
        optimizer_gen.zero_grad()
        loss_gen.backward()
        optimizer_gen.step()

    if (epoch+1) % print_every == 0:
        print(f"Epoch {epoch+1}/{num_epochs}:\t Generator loss: {loss_gen.item()}\t Discriminator loss: {loss_disc.item()}")



Epoch 10/100:	 Generator loss: 0.6969020962715149	 Discriminator loss: 1.415478229522705
Epoch 20/100:	 Generator loss: 0.7014884352684021	 Discriminator loss: 1.4510271549224854
Epoch 30/100:	 Generator loss: 0.6903109550476074	 Discriminator loss: 1.3721253871917725
Epoch 40/100:	 Generator loss: 0.7007946968078613	 Discriminator loss: 1.4018926620483398
Epoch 50/100:	 Generator loss: 0.6807171702384949	 Discriminator loss: 1.3390090465545654
Epoch 60/100:	 Generator loss: 0.7054358720779419	 Discriminator loss: 1.365788221359253
Epoch 70/100:	 Generator loss: 0.6885059475898743	 Discriminator loss: 1.4076051712036133
Epoch 80/100:	 Generator loss: 0.7028335928916931	 Discriminator loss: 1.3930323123931885
Epoch 90/100:	 Generator loss: 0.6895354986190796	 Discriminator loss: 1.4109766483306885
Epoch 100/100:	 Generator loss: 0.6818939447402954	 Discriminator loss: 1.3375091552734375


In [18]:
print("\nReal data: ")
print(data[:5])


print("\nGenerated data: ")
for _ in range(5):
    noise = torch.rand((1, seq_length))
    generated_data = generator(noise)
    # Detach the tensor and print data
    print(torch.round(generated_data).detach())


Real data: 
tensor([[0., 1., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 0., 0., 1., 1.]])

Generated data: 
tensor([[0., 0., 0., 0., 0.]])
tensor([[0., 1., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0.]])
